# 🛒 | Zava Retailer Chatbot with OpenAI Agents & Telemetry

Welcome! This notebook demonstrates how to build **Cora**, an AI shopping assistant for Zava hardware store, using Azure OpenAI Agents with full OpenTelemetry instrumentation.

Cora helps customers with:
- **Product inquiries** - Answer questions about Zava's hardware and home improvement products
- **Inventory checks** - Provide real-time stock availability information
- **Personalized discounts** - Calculate loyalty-based pricing for different customer tiers

## What You'll Learn

By the end of this notebook, you will be able to:

1. Build an AI agent with custom tools for a retail scenario
2. Configure OpenTelemetry instrumentation for GenAI observability
3. Capture GenAI-compliant spans with detailed attributes
4. Monitor agent execution including tool invocations
5. Export telemetry to Azure Monitor Application Insights
6. Analyze agent behavior through distributed tracing

Let's get started! 🚀

---

## Step 1: Verify Required Packages

This notebook requires several packages for OpenAI Agents and OpenTelemetry instrumentation:
## Step 1: Verify Required Packages

This notebook requires several packages for OpenAI Agents and OpenTelemetry instrumentation:

- `openai` - Azure OpenAI client library
- `openai-agents` - Agents framework for orchestration
- `rich` - Enhanced console output and logging
- `python-dotenv` - Environment variable management
- `opentelemetry-instrumentation-openai-agents-v2` - GenAI telemetry capture
- `azure-monitor-opentelemetry-exporter` - Export to Application Insights (optional)

**Installation Instructions:**

```bash
pip install openai openai-agents rich python-dotenv
pip install opentelemetry-instrumentation-openai-agents-v2
# Optional Azure Monitor exporter
pip install azure-monitor-opentelemetry-exporter
```

Let's verify the key packages are available:

In [ ]:
# Verify required packages are installed
import importlib.metadata

required_packages = {
    'openai': 'OpenAI SDK',
    'openai-agents': 'Agents Framework',
    'opentelemetry-sdk': 'OpenTelemetry SDK',
    'rich': 'Rich Console Output'
}

try:
    print("📦 Checking installed packages...\n")
    for package, description in required_packages.items():
        try:
            version = importlib.metadata.version(package)
            print(f"✅ {description}: {version}")
        except importlib.metadata.PackageNotFoundError:
            print(f"❌ {description} ({package}) - NOT INSTALLED")
    print("\n✅ Package verification complete!")
except Exception as e:
    print(f"❌ Error checking packages: {e}")

---

## Step 2: Verify Environment Variables

The following environment variables should be configured in your `.env` file:

**Required Variables:**
- `AZURE_OPENAI_API_KEY` - Your Azure OpenAI API key
- `AZURE_OPENAI_ENDPOINT` - Your Azure OpenAI endpoint URL
- `AZURE_OPENAI_MODEL_NAME` - The chat model deployment name (e.g., gpt-4o-mini)

**Optional Variables:**
- `AZURE_OPENAI_API_VERSION` - API version (defaults to 2024-05-01-preview)
- `APPLICATION_INSIGHTS_CONNECTION_STRING` - For Azure Monitor export

Let's check that these are set correctly:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

# Check required environment variables
required_vars = {
    'AZURE_OPENAI_API_KEY': 'Azure OpenAI API Key',
    'AZURE_OPENAI_ENDPOINT': 'Azure OpenAI Endpoint',
}

optional_vars = {
    'AZURE_OPENAI_MODEL_NAME': 'Model Name (defaults to gpt-4o-mini)',
    'AZURE_OPENAI_API_VERSION': 'API Version (defaults to 2024-05-01-preview)',
    'APPLICATION_INSIGHTS_CONNECTION_STRING': 'Application Insights (optional)'
}

print("🔍 Checking required environment variables...\n")
all_set = True
for var, description in required_vars.items():
    value = os.getenv(var)
    if value:
        # Mask sensitive values
        display_value = value[:10] + "..." if len(value) > 10 else "***"
        print(f"✅ {description}: {display_value}")
    else:
        print(f"❌ {description}: NOT SET")
        all_set = False

print("\n📋 Checking optional environment variables...\n")
for var, description in optional_vars.items():
    value = os.getenv(var)
    if value:
        display_value = value[:20] + "..." if len(value) > 20 else value
        print(f"✅ {description}: {display_value}")
    else:
        print(f"⚠️  {description}: Not set (will use default)")

if all_set:
    print("\n✅ All required environment variables are configured!")
else:
    print("\n❌ Please set the missing environment variables in your .env file")

---

## Step 3: Import Dependencies and Configure Logging

Now let's import all the libraries we'll need and set up logging:

- **OpenAI Agents**: Agent, Runner, function_tool for agent orchestration
- **OpenTelemetry**: Tracing, instrumentation, and exporters for observability
- **Rich Logging**: Enhanced console output for better debugging

In [ ]:
from __future__ import annotations

import asyncio
import logging
import os
import random
from dataclasses import dataclass
from datetime import datetime
from typing import Callable
from urllib.parse import urlparse

import openai
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
from dotenv import load_dotenv
from rich.logging import RichHandler

from opentelemetry import trace
from opentelemetry.instrumentation.openai_agents import OpenAIAgentsInstrumentor
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter

try:
    from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter
except ImportError:  # pragma: no cover - optional dependency
    AzureMonitorTraceExporter = None

load_dotenv(override=True)

# Configure rich logging
logging.basicConfig(level=logging.WARNING, format="%(message)s", datefmt="[%X]", handlers=[RichHandler()])
LOGGER = logging.getLogger("zava_retailer_chatbot")
LOGGER.setLevel(logging.INFO)

MODEL_NAME = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"
SERVICE_VERSION = "1.0.0"

print("✅ All libraries imported successfully!")

---

## Step 4: Configure OpenTelemetry Instrumentation

We'll set up helper functions to:
1. Resolve Azure OpenAI configuration
2. Enable GenAI semantic conventions for telemetry capture
3. Configure the tracer provider with Azure Monitor or console export

In [ ]:
@dataclass
class _ApiConfig:
    """Helper describing how to create the Azure OpenAI client."""

    build_client: Callable[[], object]
    model_name: str
    base_url: str
    provider: str


def _set_capture_env(provider: str, base_url: str) -> None:
    """Enable GenAI capture toggles required by the instrumentation layer."""

    capture_defaults = {
        "OTEL_GENAI_CAPTURE_MESSAGES": "true",
        "OTEL_GENAI_CAPTURE_SYSTEM_INSTRUCTIONS": "true",
        "OTEL_GENAI_CAPTURE_TOOL_DEFINITIONS": "true",
        "OTEL_GENAI_EMIT_OPERATION_DETAILS": "true",
        "OTEL_GENAI_PROVIDER_NAME": provider,
    }
    for key, value in capture_defaults.items():
        os.environ.setdefault(key, value)

    parsed = urlparse(base_url)
    if parsed.hostname:
        os.environ.setdefault("OTEL_GENAI_SERVER_ADDRESS", parsed.hostname)
    if parsed.port:
        os.environ.setdefault("OTEL_GENAI_SERVER_PORT", str(parsed.port))


def _resolve_api_config() -> _ApiConfig:
    """Return the client configuration for Azure OpenAI."""

    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"].rstrip("/")
    api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
    model_name = os.environ.get("AZURE_OPENAI_MODEL_NAME") or "gpt-4o-mini"
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    def _build_client() -> openai.AsyncAzureOpenAI:
        return openai.AsyncAzureOpenAI(
            api_version=api_version,
            azure_endpoint=endpoint,
            api_key=api_key,
        )

    return _ApiConfig(
        build_client=_build_client,
        model_name=model_name,
        base_url=endpoint,
        provider="azure.ai.openai",
    )


def _configure_tracer() -> None:
    """Configure tracer provider and exporter."""

    resource = Resource.create({
        "service.name": "zava-retailer-chatbot-service",
        "service.namespace": "ignite25",
        "service.version": SERVICE_VERSION,
    })
    provider = TracerProvider(resource=resource)
    connection_string = os.environ.get("APPLICATION_INSIGHTS_CONNECTION_STRING")

    if connection_string and AzureMonitorTraceExporter is not None:
        exporter = AzureMonitorTraceExporter.from_connection_string(connection_string)
        provider.add_span_processor(BatchSpanProcessor(exporter))
        print("[otel] Azure Monitor trace exporter configured")
    else:
        provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))
        if connection_string and AzureMonitorTraceExporter is None:
            print("[otel] Azure Monitor exporter unavailable. Install azure-monitor-opentelemetry-exporter")
        else:
            print("[otel] Console span exporter configured")

    trace.set_tracer_provider(provider)

print("✅ OpenTelemetry configuration helpers defined!")

---

## Step 5: Initialize OpenTelemetry Instrumentation

Run this cell to wire up the tracer provider and the OpenAI Agents instrumentor. This enables automatic capture of GenAI semantic conventions including:

- Agent creation and configuration
- Message exchanges between user and agent
- Tool invocations and results
- Model parameters and responses

In [ ]:
API_CONFIG = _resolve_api_config()
_set_capture_env(API_CONFIG.provider, API_CONFIG.base_url)
_configure_tracer()

OpenAIAgentsInstrumentor().instrument(tracer_provider=trace.get_tracer_provider())
CLIENT = API_CONFIG.build_client()
set_tracing_disabled(False)

print("✅ OpenTelemetry instrumentation initialized!")
print(f"📊 Provider: {API_CONFIG.provider}")
print(f"🤖 Model: {API_CONFIG.model_name}")

---

## Step 6: Define Agent Tools for Zava Chatbot

Now we'll create the custom tools that **Cora** (our Zava shopping assistant) will use to help customers. Each tool is decorated with `@function_tool` to make it available to the agent.

### Tool 1: Get Product Information

Retrieves product details by SKU including name, price, category, and description.

In [ ]:
# Simulated product database (in production, this would query Azure AI Search or a database)
ZAVA_PRODUCTS = {
    "PFIP000002": {
        "name": "Interior Eggshell Paint",
        "price": 44.0,
        "category": "PAINT & FINISHES",
        "subcategory": "INTERIOR PAINT",
        "stock": 80,
        "description": "Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup."
    },
    "PFIP000001": {
        "name": "Premium Interior Latex Flat",
        "price": 40.0,
        "category": "PAINT & FINISHES",
        "subcategory": "INTERIOR PAINT",
        "stock": 19,
        "description": "High-quality flat interior paint with excellent coverage and hide, perfect for ceilings and low-traffic areas."
    },
    "PTDR000001": {
        "name": "Cordless Drill 18V Li-Ion",
        "price": 115.0,
        "category": "POWER TOOLS",
        "subcategory": "DRILLS",
        "stock": 3,
        "description": "Professional cordless drill with lithium-ion battery, variable speed control, and LED work light."
    },
    "HTHM041300": {
        "name": "Finishing Hammer 13oz",
        "price": 25.0,
        "category": "HAND TOOLS",
        "subcategory": "HAMMERS",
        "stock": 75,
        "description": "Lightweight finishing hammer with smooth face for trim work and delicate construction tasks."
    }
}


@function_tool
def get_product_info(sku: str) -> dict[str, object]:
    """Retrieves detailed product information from the Zava catalog by SKU."""
    
    LOGGER.info("Looking up product info for SKU: %s", sku)
    
    if sku in ZAVA_PRODUCTS:
        product = ZAVA_PRODUCTS[sku]
        return {
            "sku": sku,
            "name": product["name"],
            "price": product["price"],
            "category": product["category"],
            "subcategory": product["subcategory"],
            "description": product["description"],
            "available": True
        }
    else:
        return {
            "sku": sku,
            "available": False,
            "message": f"Product {sku} not found in catalog"
        }

print("✅ Tool 1 defined: get_product_info()")

### Tool 2: Check Inventory

Checks current stock levels and provides status indicators (Out of stock, Low stock, In stock, Well stocked).

### Step 4: Register agent tools and construct the Zava chatbot agent

With instrumentation active, declare the reusable tools that "Cora" (the Zava shopping assistant) will use to help customers:

1. **get_product_info**: Retrieves detailed information about Zava products by SKU or category
2. **check_inventory**: Checks current stock levels for specific products
3. **calculate_discount**: Computes personalized discounts based on customer loyalty tier and cart value

These tools simulate the backend services that a real chatbot would integrate with.


In [ ]:
# Simulated product database (in real implementation, this would query Azure AI Search or a database)
ZAVA_PRODUCTS = {
    "PFIP000002": {
        "name": "Interior Eggshell Paint",
        "price": 44.0,
        "category": "PAINT & FINISHES",
        "subcategory": "INTERIOR PAINT",
        "stock": 80,
        "description": "Durable eggshell finish paint with subtle sheen, ideal for living rooms and bedrooms with easy cleanup."
    },
    "PFIP000001": {
        "name": "Premium Interior Latex Flat",
        "price": 40.0,
        "category": "PAINT & FINISHES",
        "subcategory": "INTERIOR PAINT",
        "stock": 19,
        "description": "High-quality flat interior paint with excellent coverage and hide, perfect for ceilings and low-traffic areas."
    },
    "PTDR000001": {
        "name": "Cordless Drill 18V Li-Ion",
        "price": 115.0,
        "category": "POWER TOOLS",
        "subcategory": "DRILLS",
        "stock": 3,
        "description": "Professional cordless drill with lithium-ion battery, variable speed control, and LED work light."
    },
    "HTHM041300": {
        "name": "Finishing Hammer 13oz",
        "price": 25.0,
        "category": "HAND TOOLS",
        "subcategory": "HAMMERS",
        "stock": 75,
        "description": "Lightweight finishing hammer with smooth face for trim work and delicate construction tasks."
    }
}


@function_tool
def get_product_info(sku: str) -> dict[str, object]:
    """Retrieves detailed product information from the Zava catalog by SKU."""
    
    LOGGER.info("Looking up product info for SKU: %s", sku)
    
    if sku in ZAVA_PRODUCTS:
        product = ZAVA_PRODUCTS[sku]
        return {
            "sku": sku,
            "name": product["name"],
            "price": product["price"],
            "category": product["category"],
            "subcategory": product["subcategory"],
            "description": product["description"],
            "available": True
        }
    else:
        return {
            "sku": sku,
            "available": False,
            "message": f"Product {sku} not found in catalog"
        }


@function_tool
def check_inventory(sku: str) -> dict[str, object]:
    """Checks current stock levels for a specific product SKU."""
    
    LOGGER.info("Checking inventory for SKU: %s", sku)
    
    if sku in ZAVA_PRODUCTS:
        stock = ZAVA_PRODUCTS[sku]["stock"]
        in_stock = stock > 0
        
        # Determine stock status
        if stock == 0:
            status = "Out of stock"
        elif stock < 10:
            status = "Low stock"
        elif stock < 50:
            status = "In stock"
        else:
            status = "Well stocked"
        
        return {
            "sku": sku,
            "product_name": ZAVA_PRODUCTS[sku]["name"],
            "stock_level": stock,
            "in_stock": in_stock,
            "status": status
        }
    else:
        return {
            "sku": sku,
            "in_stock": False,
            "message": f"Product {sku} not found"
        }

print("✅ Tool 2 defined: check_inventory()")

### Tool 3: Calculate Discount

Computes personalized discounts based on customer loyalty tier (Bronze, Silver, Gold, Platinum) and cart total, with bonus discounts for large orders.

In [ ]:
    """
    Calculates personalized discount based on customer loyalty tier and cart value.
    
    Args:
        customer_tier: Customer loyalty tier ('bronze', 'silver', 'gold', 'platinum')
        cart_total: Total cart value in dollars
    """
    
    LOGGER.info("Calculating discount for tier: %s, cart total: $%.2f", customer_tier, cart_total)
    
    # Base discount by tier
    tier_discounts = {
        "bronze": 0.05,    # 5%
        "silver": 0.10,    # 10%
        "gold": 0.15,      # 15%
        "platinum": 0.20   # 20%
    }
    
    base_discount = tier_discounts.get(customer_tier.lower(), 0.0)
    
    # Additional discount for large orders
    if cart_total >= 500:
        bonus_discount = 0.05  # Extra 5% for orders over $500
    elif cart_total >= 250:
        bonus_discount = 0.03  # Extra 3% for orders over $250
    else:
        bonus_discount = 0.0
    
    total_discount = min(base_discount + bonus_discount, 0.30)  # Cap at 30%
    discount_amount = cart_total * total_discount
    final_price = cart_total - discount_amount
    
    return {
        "customer_tier": customer_tier,
        "cart_total": cart_total,
        "base_discount_percent": base_discount * 100,
        "bonus_discount_percent": bonus_discount * 100,
        "total_discount_percent": total_discount * 100,
        "discount_amount": round(discount_amount, 2),
        "final_price": round(final_price, 2)
    }

print("✅ Tool 3 defined: calculate_discount()")

---

## Step 7: Create the Cora Agent

Now we'll assemble all the pieces into a complete AI agent:

- **Name**: "Cora - Zava Shopping Assistant"
- **Instructions**: Define the agent's behavior and personality
- **Tools**: The three custom tools we defined above
- **Model**: Azure OpenAI chat completion model

In [ ]:
AGENT = Agent(
    name="Cora - Zava Shopping Assistant",
    instructions=(
        "You are Cora, a polite and helpful AI shopping assistant for Zava, a hardware and home improvement retailer. "
        "Your role is to:\n"
        "1. Answer customer questions about Zava products in a friendly, professional tone\n"
        "2. Provide accurate product information including prices, descriptions, and availability\n"
        "3. Check inventory levels and inform customers about stock status\n"
        "4. Calculate personalized discounts for customers based on their loyalty tier\n"
        "5. Help customers make informed purchasing decisions\n\n"
        "Always be courteous, factual, and helpful. Use the available tools to retrieve accurate information. "
        "If a product is out of stock, suggest similar alternatives when possible."
    ),
    tools=[get_product_info, check_inventory, calculate_discount],
    model=OpenAIChatCompletionsModel(model=API_CONFIG.model_name, openai_client=CLIENT),
)

print("✅ Agent created successfully!")
print(f"🤖 Agent Name: {AGENT.name}")
print(f"🔧 Tools Available: {[tool.name for tool in AGENT.tools]}")

---

## Step 8: Run the Zava Chatbot Agent

Execute the cell below to simulate a customer interaction with Cora. The agent will:

1. **Receive** the customer's question about interior paint and discounts
2. **Decide** which tools to invoke based on the query
3. **Execute** tool calls to retrieve product and discount information
4. **Generate** a helpful, personalized response

**What to Watch For:**

In the console or Application Insights, you'll see spans for:
- `zava_customer_session` - The overall customer interaction
- `create_agent` - Agent initialization
- `invoke_agent` - Agent processing and reasoning
- `execute_tool` - Each tool invocation (product lookup, inventory check, discount calculation)

These spans include rich attributes following GenAI semantic conventions!

In [ ]:
async def run_zava_chatbot():
    tracer = trace.get_tracer(__name__)
    with tracer.start_as_current_span(f"zava_customer_session[{API_CONFIG.provider}]") as span:
        # Simulate a customer inquiry
        user_request = "Hi! I'm looking for interior paint for my living room. What do you have available? Also, I'm a Gold member with about $200 in my cart - what kind of discount can I get?"
        
        span.set_attribute("user.request", user_request)
        span.set_attribute("gen_ai.provider.name", API_CONFIG.provider)
        span.set_attribute("gen_ai.request.model", API_CONFIG.model_name)
        span.set_attribute("agent.name", AGENT.name)
        span.set_attribute("customer.tier", "gold")
        span.set_attribute("cart.value", 200.0)
        span.set_attribute("product.category", "paint")
        
        try:
            result = await Runner.run(AGENT, input=user_request)
            output = result.final_output or ""
            span.set_attribute("agent.response", output[:500])
            span.set_attribute("request.success", True)
            
            print("="*80)
            print("CORA (Zava Shopping Assistant):")
            print("="*80)
            print(output)
            print("="*80)
            
        except Exception as exc:
            span.record_exception(exc)
            span.set_attribute("request.success", False)
            raise

await run_zava_chatbot()
trace.get_tracer_provider().shutdown()

print("\n✅ Session complete! Check your telemetry export destination for detailed traces.")

---

## Step 9: Review Results and Next Steps

Congratulations! 🎉 You've successfully:

1. ✅ Built an AI agent with custom tools for a retail scenario
2. ✅ Configured OpenTelemetry instrumentation for GenAI observability
3. ✅ Captured GenAI-compliant spans with detailed attributes
4. ✅ Monitored agent execution including tool invocations
5. ✅ Exported telemetry to Azure Monitor or console

### Understanding the Telemetry

The spans captured include:

**Session Span** (`zava_customer_session`):
- User request and agent response
- Customer tier and cart value
- Overall success status

**Agent Spans**:
- Agent creation with model and tools configuration
- Agent invocation with reasoning steps
- System instructions and prompts

**Tool Spans**:
- Tool invocations with parameters
- Tool results and execution time
- Error handling and exceptions

### View in Application Insights

If you configured `APPLICATION_INSIGHTS_CONNECTION_STRING`, you can:

1. Open the Azure Portal
2. Navigate to your Application Insights resource
3. Go to "Transaction search" or "Application map"
4. Filter for traces from "zava-retailer-chatbot-service"
5. Click on a trace to see the full distributed trace with all spans

### Try These Experiments:

**Experiment 1: Add More Products**
```python
# Expand the ZAVA_PRODUCTS dictionary with more items from /labs/data/products.csv
```

**Experiment 2: Integrate Azure AI Search**
```python
# Replace the simulated product database with real Azure AI Search queries
# Use the AzureAISearchTool similar to notebook 11-agent-service-creation.ipynb
```

**Experiment 3: Add More Tools**
```python
@function_tool
def get_product_recommendations(category: str, price_range: tuple) -> list[dict]:
    """Get product recommendations based on category and price range."""
    # Implementation here
    pass
```

**Experiment 4: Test Different Scenarios**
- Try different customer loyalty tiers (Bronze, Silver, Platinum)
- Ask about out-of-stock products
- Request multiple products in one query
- Test error handling with invalid SKUs

**Experiment 5: Analyze Telemetry**
- Compare trace duration for different queries
- Identify which tools are called most frequently
- Analyze agent reasoning patterns
- Monitor for errors or performance issues


---

## Related Notebooks

Continue exploring agent tracing with different frameworks:

- **`51-openai-weekend-planner.ipynb`** - Original weekend planning agent example
- **`52-langchain-weekend-planner.ipynb`** - LangChain implementation with tracing
- **`53-langgraph-music_router.ipynb`** - LangGraph routing example with observability

### Learn More

- [Azure AI Foundry Agents Documentation](https://learn.microsoft.com/azure/ai-foundry/agents/overview)
- [OpenTelemetry for GenAI](https://opentelemetry.io/docs/languages/python/)
- [Azure Monitor OpenTelemetry](https://learn.microsoft.com/azure/azure-monitor/app/opentelemetry-enable)
- [GenAI Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/)

---

## Appendix: Understanding OpenTelemetry and Tracing

This section provides beginner-friendly explanations of key concepts used in this notebook.

### What is OpenTelemetry?

OpenTelemetry is an open-source framework for collecting telemetry data (metrics, logs, and traces) from your applications. Think of it as a standardized way to answer: "What is my application doing right now, and how is it performing?"

**Key benefits:**
- **Vendor-neutral**: Works with many monitoring tools, not locked to one provider
- **Standardized**: Uses common terminology and data formats
- **Built-in**: Supported natively in modern cloud platforms like Azure

### Core Tracing Concepts

#### 1. Trace

A **trace** represents the complete journey of a single request through your system.

**Example**: When a customer asks Cora "What paint do you have?", the trace captures:
- The customer's question arriving
- The agent deciding which tools to call
- Each tool execution (product lookup, inventory check)
- The final response generation

Think of a trace as the full story of one customer interaction.

#### 2. Span

A **span** is a single operation within a trace. Spans can be nested (parent-child relationships) to show how work flows through your system.

**Example spans in this notebook:**
- `zava_customer_session` - The overall customer interaction (parent span)
- `invoke_agent` - Agent processing the request (child span)
- `execute_tool: get_product_info` - Looking up product details (child span)
- `execute_tool: calculate_discount` - Computing discount (child span)

**Analogy**: If a trace is a recipe, spans are the individual steps (preheat oven, mix ingredients, bake).

#### 3. Attributes

**Attributes** are key-value pairs that provide context about what happened in a span.

**Examples from this notebook:**
```python
span.set_attribute("user.request", "What paint do you have?")
span.set_attribute("customer.tier", "gold")
span.set_attribute("agent.name", "Cora")
span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
```

Attributes let you filter and analyze traces: "Show me all failed requests from Gold tier customers."

#### 4. Trace ID

Every trace gets a unique identifier (TraceID). This lets you:
- Find all spans belonging to one customer interaction
- Correlate logs and metrics with traces
- Debug specific issues by trace ID

### Azure AI Foundry Tracing Features

Azure AI Foundry provides specialized tracing for AI agents with these capabilities:

#### Automatic Instrumentation

When you use `OpenAIAgentsInstrumentor().instrument()`, the framework automatically captures:
- **Agent creation**: Model, instructions, tools configured
- **Agent invocations**: User messages, system prompts, reasoning steps
- **Tool executions**: Which tools were called, with what parameters, what they returned
- **Model calls**: Token usage, latency, responses

You don't need to manually create spans for these operations.

#### GenAI Semantic Conventions

OpenTelemetry defines standard attribute names for AI applications. This notebook uses:

**For agents:**
- `gen_ai.provider.name` - Which AI provider (e.g., "azure.ai.openai")
- `gen_ai.request.model` - Model name (e.g., "gpt-4o-mini")
- `agent.name` - Agent identifier

**For operations:**
- `user.request` - Customer's question
- `agent.response` - Agent's answer
- `request.success` - Did it work (true/false)

These conventions ensure your traces are readable across different tools.

#### Application Insights Integration

**Application Insights** is Azure's monitoring service. When you configure:
```python
APPLICATION_INSIGHTS_CONNECTION_STRING = "..."
```

Your traces automatically flow to Azure Monitor where you can:
- **Search traces**: Find specific customer interactions
- **Build dashboards**: Visualize agent performance over time
- **Set alerts**: Get notified when errors spike
- **Analyze trends**: Track response times, tool usage patterns

### How Tracing Helps You

#### 1. Debugging

When something goes wrong, traces show you exactly what happened:
```
Customer asked about paint
  ✓ Agent invoked successfully
  ✓ Tool: get_product_info("PFIP000002") → Found product
  ✗ Tool: calculate_discount("gold", 200) → Error: Invalid tier format
  ✗ Agent failed to generate response
```

You can see the tool call failed and why.

#### 2. Performance Optimization

Traces show how long each operation takes:
```
Total request: 2.3 seconds
  - Agent reasoning: 0.8 seconds
  - Tool: get_product_info: 0.1 seconds
  - Tool: check_inventory: 1.2 seconds ← SLOW!
  - Response generation: 0.2 seconds
```

Now you know to optimize the inventory check.

#### 3. Understanding Agent Behavior

Traces reveal how your agent makes decisions:
- Which tools does it call most often?
- Does it call tools in the right order?
- Are some tools never used?
- How does it handle ambiguous questions?

This helps you refine instructions and tool configurations.

### Best Practices

#### 1. Use Descriptive Span Names

```python
# Good
with tracer.start_as_current_span("calculate_gold_tier_discount"):

# Less helpful
with tracer.start_as_current_span("calc"):
```

#### 2. Add Meaningful Attributes

```python
# Good - provides context
span.set_attribute("customer.tier", "gold")
span.set_attribute("cart.value", 200.0)
span.set_attribute("discount.amount", 30.0)

# Less useful
span.set_attribute("data", "some value")
```

#### 3. Don't Log Sensitive Data

```python
# Bad - contains PII
span.set_attribute("customer.email", "john@example.com")
span.set_attribute("customer.credit_card", "1234-5678...")

# Good - use IDs
span.set_attribute("customer.id", "cust_12345")
```

#### 4. Set Trace Sampling for Production

In production, you may not want to trace every request (too much data). Configure sampling:
```python
# Trace 10% of requests
sampler = TraceIdRatioBased(0.1)
```

### Terminology Quick Reference

| Term | Simple Definition |
|------|-------------------|
| **Trace** | The full story of one request through your system |
| **Span** | A single step or operation within a trace |
| **Attribute** | Extra information about what happened (key-value pair) |
| **Tracer** | The object you use to create spans |
| **Exporter** | Sends trace data to a monitoring system |
| **Instrumentation** | Code that automatically creates spans |
| **Semantic Convention** | Standard names for common attributes |
| **Distributed Tracing** | Following a request across multiple services |

### Further Reading

For deeper understanding:

- **[OpenTelemetry Concepts](https://opentelemetry.io/docs/concepts/)** - Official OTel documentation
- **[Azure AI Foundry Tracing Guide](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-agents-sdk)** - Azure-specific tracing setup
- **[Application Insights Documentation](https://learn.microsoft.com/azure/azure-monitor/app/distributed-tracing)** - Using Azure Monitor for traces
- **[GenAI Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/)** - Standard attributes for AI applications